In [71]:
# import necessary packages
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

# supress scientific notation in printouts
pd.options.display.float_format = '{:.2f}'.format

In [75]:
df = pd.read_csv('/home/ryank/DA Projects/DA_datafiles/beer_reviews.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586614 non-null  int64  
 1   brewery_name        1586599 non-null  object 
 2   review_time         1586614 non-null  int64  
 3   review_overall      1586614 non-null  float64
 4   review_aroma        1586614 non-null  float64
 5   review_appearance   1586614 non-null  float64
 6   review_profilename  1586266 non-null  object 
 7   beer_style          1586614 non-null  object 
 8   review_palate       1586614 non-null  float64
 9   review_taste        1586614 non-null  float64
 10  beer_name           1586614 non-null  object 
 11  beer_abv            1518829 non-null  float64
 12  beer_beerid         1586614 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB
None


In [8]:
df.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
count,1586614.00,1586614.00,1586614.00,1586614.00,1586614.00,1586614.00,1586614.00,1518829.00,1586614.00
mean,3130.10,1224089280.01,3.82,3.74,3.84,3.74,3.79,7.04,21712.79
std,5578.10,76544274.54,0.72,0.70,0.62,0.68,0.73,2.32,21818.34
min,1.00,840672001.00,0.00,1.00,0.00,1.00,1.00,0.01,3.00
25%,143.00,1173224188.25,3.50,3.50,3.50,3.50,3.50,5.20,1717.00
50%,429.00,1239202881.50,4.00,4.00,4.00,4.00,4.00,6.50,13906.00
75%,2372.00,1288568405.25,4.50,4.00,4.00,4.00,4.50,8.50,39441.00
max,28003.00,1326285348.00,5.00,5.00,5.00,5.00,5.00,57.70,77317.00


In [22]:
# count number of unique beers and breweries
print('Number of unique breweries by ids:', df['brewery_id'].nunique())
print('Number of unique beers by ids:', df['beer_beerid'].nunique())
print('Number of unique beer styles by ids:', df['beer_style'].nunique())
print('Number of unique beer reviewers: ', df['review_profilename'].nunique())

Number of unique breweries by ids: 5840
Number of unique beers by ids: 66055
Number of unique beers by ids: 104
Number of unique beer reviewers:  33387


In [63]:
### data wrangling

# reformat review time to date format
df['review_time'] = pd.to_datetime(df['review_time']).dt.date

## check and handle null values
print(df.isnull().sum())

# remove 15 rows where brewery name is null
df = df[df['brewery_name'].notna()]

# replace null ABV's with average abv for its given beer style
df['beer_abv'] = df['beer_abv'].fillna(df.groupby('beer_style')['beer_abv'].transform('mean'))
df['reviewer_duplicates'] = df['review_profilename'] + df['beer_name']

brewery_id                0
brewery_name             15
review_time               0
review_overall            0
review_aroma              0
review_appearance         0
review_profilename      348
beer_style                0
review_palate             0
review_taste              0
beer_name                 0
beer_abv              67785
beer_beerid               0
dtype: int64


In [70]:
## handle duplicate values

# remove reviews where reviewer rated a beer multiple times
print(len(df))
df1 = df.drop_duplicates(subset=['reviewer_duplicates'], keep='last')
dfdup = df[df.duplicated(subset=['reviewer_duplicates'], keep=False)].sort_values('reviewer_duplicates')
print(len(dfdup))
print(len(df1))
dfdup.head(10)

1586599
46083
1561391


,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,reviewer_duplicates
494164,132,Rogue Ales,1219030990,4.50,4.00,4.50,100floods,Russian Imperial Stout,4.50,4.50,Imperial Stout,11.00,356,100floodsImperial Stout
742224,7730,Nøgne Ø - Det Kompromissløse Bryggeri A/S,1221703466,4.50,3.00,5.00,100floods,Russian Imperial Stout,3.50,4.00,Imperial Stout,9.00,28248,100floodsImperial Stout
93878,140,Sierra Nevada Brewing Co.,1290908318,4.50,3.50,3.00,100floods,American IPA,3.00,4.50,Sierra Nevada Torpedo Extra IPA,7.20,30420,100floodsSierra Nevada Torpedo Extra IPA
93758,140,Sierra Nevada Brewing Co.,1298151357,3.00,2.00,4.00,100floods,American IPA,3.50,3.00,Sierra Nevada Torpedo Extra IPA,7.20,30420,100floodsSierra Nevada Torpedo Extra IPA
84180,140,Sierra Nevada Brewing Co.,1201010077,5.00,5.00,4.50,11osixBrew,American IPA,4.50,4.50,Sierra Nevada Celebration Ale,6.80,1904,11osixBrewSierra Nevada Celebration Ale
84178,140,Sierra Nevada Brewing Co.,1201187157,4.00,4.50,4.50,11osixBrew,American IPA,4.00,4.00,Sierra Nevada Celebration Ale,6.80,1904,11osixBrewSierra Nevada Celebration Ale
603196,696,Thomas Hooker Brewing Company,1203877292,4.00,4.50,4.50,11osixBrew,Doppelbock,4.00,3.50,Thomas Hooker Liberator Doppelbock,8.00,15589,11osixBrewThomas Hooker Liberator Doppelbock
603217,696,Thomas Hooker Brewing Company,1194268558,5.00,4.50,4.00,11osixBrew,Doppelbock,4.50,4.50,Thomas Hooker Liberator Doppelbock,8.00,15589,11osixBrewThomas Hooker Liberator Doppelbock
1481146,124,Paulaner Brauerei GmbH & Co. KG,1285722499,4.00,3.00,3.00,1759Girl,Märzen / Oktoberfest,4.00,4.00,Paulaner Oktoberfest-Märzen,6.00,386,1759GirlPaulaner Oktoberfest-Märzen
1481277,124,Paulaner Brauerei GmbH & Co. KG,1250176060,3.50,3.00,5.00,1759Girl,Märzen / Oktoberfest,4.00,3.50,Paulaner Oktoberfest-Märzen,6.00,386,1759GirlPaulaner Oktoberfest-Märzen
